In [1]:
vagrantStructPath = "config/vagrant.txt"
VMStructPath = "config/VMbasic.txt"
bootstrapStructPath = "config/bootstrap.txt"
simCmdPath = "config/eStationSimCmd.txt"

with open(vagrantStructPath) as f:
    vangrantStructure = f.read()

with open(VMStructPath) as f:
    VMStructure = f.read()

with open(bootstrapStructPath) as f:
    bootstrapStructure = f.read()

with open(simCmdPath) as f:
    simCmd = f.read()

In [8]:
def outputSimCmd(ID, schURL, sch, server, PCS, PWCATL, PWCEV, BESS, EVSE):
    keys = {
        "{eStationID}": ID,
        "{schemaRegistryURL}": schURL,
        "{schemaPath}": sch,
        "{serverIP}": server,
        "{nPCS}": str(PCS),
        "{nPWCATLs}": str(PWCATL),
        "{nPWCEVs}": str(PWCEV),
        "{BESSs}": str(BESS),
        "{nEVSEs}": str(EVSE)
    }
    cmd = simCmd
    for key, val in keys.items():
        cmd = cmd.replace(key, val)
    return cmd

def outputBootstrap(cmds):
    return bootstrapStructure.replace("{programCmds}", cmds)

def outputVM(name, ref, bootFile):
    output = VMStructure.replace("{VMname}", name)
    output = output.replace("{VMRef}", ref)
    return output.replace("{bootstrapFileName}", bootFile)

def outputVagrant(config):
    return vangrantStructure.replace("{config}", config)

In [7]:
#Parameters of the python sim code
schemaRegistryURL = "http://192.168.188.54:8081"
schemaPath = "eStationAvroSchema.avsc"
bootstrap = "192.168.188.54:9092"
PCS = 1 
PWCATL = 1 
PWCEV = 2 
BESS = 1 
EVSE = 8

#N sims
SimPerVM = 2
nVMs = 2

VMs = []
for i in range(nVMs):
    VMname = f"VM{i}"
    VMRef = f"eStation{i}"
    commands = []
    for n in range(SimPerVM):
        eStationID = f"eStation_{i}_{n}"
        commands += [outputSimCmd(eStationID, schemaRegistryURL, schemaPath, bootstrap, PCS, PWCATL, PWCEV, BESS, EVSE)]
    bootstrapFile = outputBootstrap("\n\n".join(commands))
    bootstrapName = f"bootstrap_{i}.sh"
    with open(f"test/{bootstrapName}", 'w') as f:
        f.write(bootstrapFile)
    VMs += [outputVM(VMname, VMRef, bootstrapName)]
    
vagrant = outputVagrant("\n\n".join(VMs))

with open('test/Vagrantfile', 'w') as f:
    f.write(vagrant)

In [13]:
output = outputSimCmd("test", schemaRegistryURL, schemaPath, bootstrap, PCS, PWCATL, PWCEV, BESS, EVSE)

In [14]:
with open('test/readme.txt', 'w') as f:
    f.write(output)